# 토큰화 알고리즘 다루기

* 토큰화는 텍스트 입력을 토큰으로 분ㄴ할하고 각 토큰에 식별자를 할당하여 신경망에 전달하는 것
* 가장 직관적인 방법은 공백(whitespace) 문자를 기준으로 시퀀스를 작은 조각으로 나누는 것
* 그러나 이런 방식은 일본어 등 일부 언어에 적합하지 않음. 게다가 어휘 크기가 지나치게 커지는 문제를 일으킬 수 있음. 거의 모든 트랜스포머 모델은 부분단어 토큰화를 사용하는데, 이는 차원 축소뿐만 아니라 훈련 과정에서 보지 못한 희귀 단어나 미지의 단어를 인코딩하기 위해서 사용함
* 토큰화는 모든 단어, 특히 희귀 단어나 미지의 단어를 훈력 말뭉치의 곳곳에 있는 의미 있는 짧은 기호 조각들로 분해할 수 있다는 아이디어에 기반함

* 전통적인 토크나이저는 고도의 규칙 기반(rule-based) 기법으로 만들어 졌음. 반면에 트랜스포머와 함께 쓰는 토큰화 알고리즘은 자기 지도 학습(self-supervised learning)에 기반해서 말뭉치에서 규칙들을 추출함
* 간단하고 직관적인 규칙 기반 토큰화 방법은 문자, 구두점(문장부호), 공백 단위로 토큰을 생성함. 하지만 이런 문자 기반 토큰화를 사용하면 언어 모델이 입력의 의미를 놓치게 됨
* 최근 BPE와 같은 여러 고급 부분단어 토큰화 알고리즘이 트랜스포머 아키텍처의 중요한 부분이 됨. 이러한 현대적 토큰화 절차는 두 단계로 구성됨
    * 첫 단계는 사전 토큰화(pre-tokenization)인데, 여기서는 공백 문자나 언어별 규칙을 이용해서 간단한 방식으로 입력 토큰을 분할함
    * 둘째 단계는 토크나이저 학습임. 이 단계에서 생성된 토큰들에 기반해서 적절한 크기의 기본 어휘가 구축됨


## BPE

* BPE(byte-pair encoding; 바이트 쌍 인코딩)는 일종의 데이터 압축 기술임. 이 토큰화 알고리즘은 데이터 시퀀스를 훑어서 자주 등장하는 바이트 쌍을 하나의 기호로 대체하는 작업을 반복함. BPE는 Neural machine translation of rare words with subword units (Sennrich et al., 2015)에서 처음 제안함. 기계번역에서 미지 단어와 희소 단어 문제를 해결하기 위한 것으로 이제는 GPT-2를 비롯한 여러 최신 모델에 성공적으로 쓰이고 있음
* BPE는 주어진 텍스트를 일련의 문자 n-그램으로 표현함. 문자 n-그램을 문자 수준 부분단어(character-level subword)라고 부름. 훈련은 말뭉치에 등장한 모든 유니코드 문자(또는 기호)로 구성된 어휘로 시작함
* 영어의 경우에는 이 어휘가 작지만 한국어처럼 문자가 다양한 언어에서 상당히 클 수 있음. 주어진 어휘에 기반해서 BPE는 문자 바이그램(2-그램)을 계산해서 가장 자주 나오는 것들을 특별한 새 기호로 대체하는 과정을 반복함
* 예를 들어 영어 텍스트에는 t와 h의 쌍이 자주 나오므로, BPE는 이 쌍을 하나의 기호로 대체함. 이러한 과정을 어휘가 원하는 크기가 될 떄까지 반복함. 가장 흔히 쓰이는 어휘 크기는 30,000임
* BPE는 특히 미지 단어(unknown word), 즉 이전에 본적이 없는 단어를 표현하는 데 효과적임. 하지만 희소 단어나 희소 부분단어가 포함된 단어를 제대로 처리한다는 보장은 없음. 희소 단어의 경우 BPE는 희소한 문자들을 특수 기호 <UNK>와 연관시키기 때문에 단어의 의미가 조금 손실될 수 있음
* 이에 대한 잠재적 해결책으로 바이트 수준 BPE(byte-level BPE, BBPE)가 제안됨. BBPE는 유니코드 문자 대신 256바이트 어휘 집합을 이용해서 모든 기본 문자가 어휘에 포함되게 함

## WordPiece 토큰화

* WordPiece도 인기 있는 단어 분할 알고리즘으로, BERT, DistilBERT, ELECTRA 등 여러 모델에 쓰임
* 이 알고리즘은 2012년 슈스터(Schuster)와 나카지마(Nakajima)가 일본어와 한국어 음성 문제를 해결하기 위해 제안함
* 그들은 영어에서는 단어 분할(word segmentation)이 크게 중요하지 않지만, 공백이 별로 쓰이지 않는 아시아 언어의 처리에는 단어 분할이 중요한 전처리 과정이라는 점에 착안해서 연구를 진행했음. 실제로 아시아 언어의 NLP 연구에서는 단어 분할 접근 방식을 자주 볼 수 있음. 
* BPE처럼 WordPiece도 대규모 말뭉치를 이용해서 어휘와 병합 규칙(merging rule)을 학습함. BPE와 BBPE는 공동 출현 통계에 기반해서 병합 규칙을 학습하는 반면, WordPiece 알고리즘은 최대 가능도(maximum likelihood; 최대 우도) 추정을 이용해서 말뭉치에서 병합 규칙을 추출함
* 먼저 유니코드 문자(어휘 기호(vocabulary symbol)라고도 함)로 어휘를 초기함
* 그런 다음 훈련 말뭉치의 각 단어를 기호 목록(초기에는 유니코드 문자들)으로 취급해서 가능한 모든 기호 쌍에 대해 가능도(빈도가 아니라)의 최대화에 기반해 두 기호를 병합해서 새로운 기호를 생성함
* 이러한 생성 과정을 어휘가 원하는 크기가 될 때까지 반복함

## SentencePiece 토큰화

* 앞서 소개한 토큰화 알고리즘들은 텍스트를 공백으로 구분된 단어 목록으로 취급함. 하지만 이런 공백 기반 분할이 통하지 않는 언어들도 있음. 예를 들어 일부 언어는 둘 이상의 단어로 이루어진 복합명사를 공백 없이 붙여 쓴다(이를테면 한국어의 정보통신기술이나 독일어의 Menschenrechte(인권))
* 이에 대한 해결책은 언어별 사전 토큰화기를 사용하는 것. 독일어의 경우 NLP 파이프라인에서 복합어 분할 모듈을 이용해서 단어를 더 작은 조각으로 분할할 수 있는지 확인하는 식으로 이 문제를 해결할 수 있음
* 그러나 띄어쓰기가 아예 없는, 그러니까 단어와 단어 사이에 공백을 아예 두지 않는 언어들도 있음. 예를 들어 중국어, 일본어, 태국어 같은 동아시아 언어들이 그렇다. 이런 공백의 한계를 극복하기 위해 설계된 것이 SentencePiece 알고리즘임
* 2018년에 구도(Kudo) 등이 제안한 SentencePiece는 간단하면서도 언어 독립적인 토큰화 알고리즘임. 이 알고리즘은 입력 텍스트를 원시 입력 스트림으로 취급하는데, 공백 문자도 문자 집합의 일부로 간주한다는 점이 특징임
* SentencePiece 알고리즘을 이용하는 토크나이저는 the_ character 같은 출력을 생성함
* ALBERT, XLNet, Marina, T5 등 인기 있는 언어모델들이 SentencePiece를 사용함

## 토큰화 파이프라인

* 정규화(normalization) 단계는 소문자 변환, 공백 제거, 유니코드 정규화, 악센트 기호 제거 같은 기본적인 텍스트 정리 작업을 수행함
* 사전 토큰화 단계는 그 다음의 모델 훈력 단계를 위한 말뭉치를 준비한다. 특히, 띄어쓰기(공백) 등의 규칙에 따라 입력을 토큰들로 분할한다.
* 모델 학습 단계는 앞에서 논의한 BPE, BBPE, WordPiece 같은 부분단어 토큰화 알고리즘을 적용해서 토크나이저를 훈력한다. 이 단계에서는 부분단어/어휘가 발견되고, 생성 규칙이 학습된다.
* 후처리(post-processing) 단계는 Transformers 라이브러리의 트랜스포머 모델 클래스들과 호환되는 BertProcessor 같은 고급 클래스 구성요소를 제공한다. 후처리 단계에서는 주로 토큰화된 입력에 [CLS]나 [SEP] 같은 특수 토큰을 추가한다. 후처리 단계를 거친 토큰들이 트랜스포머 아키텍처에 입력된다.
* 디코딩 단계는 토큰 ID들을 원래의 문자열로 변환한다. 주로 진행 상황을 조사하는 데 쓰인다.

## Loading a Turkish Pre-trained Tokenizer

In [2]:
from transformers import AutoModel, AutoTokenizer

tokenizer_tur = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased",)
print(f"VOC size is: {tokenizer_tur.vocab_size}")
print(f"The model is {type(tokenizer_tur)}")

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base",)
print(f"VOC size is: {tokenizer.vocab_size}")
print(f"The model is {type(tokenizer)}")

VOC size is: 32000
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
VOC size is: 32000
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


## Loading an English Pre-trained Tokenizer

In [3]:
from transformers import AutoModel, AutoTokenizer

tokenizer_en = AutoTokenizer.from_pretrained("bert-base-uncased")
print(f"VOC size is: {tokenizer_en.vocab_size}")
print(f"The model is {type(tokenizer_en)}")

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base",)
print(f"VOC size is: {tokenizer.vocab_size}")
print(f"The model is {type(tokenizer)}")

VOC size is: 30522
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
VOC size is: 32000
The model is <class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [4]:
word_en = "telecommunications"
print(f"is in Turkish Model ? {word_en in tokenizer_tur.vocab}")
print(f"is in English Model ? {word_en in tokenizer_en.vocab}")

is in Turkish Model ? False
is in English Model ? True


In [5]:
print(tokenizer_en.tokenize(word_en))
print(tokenizer_tur.tokenize(word_en))
print(tokenizer.tokenize(word_en))

['telecommunications']
['tel', '##eco', '##mm', '##un', '##ica', '##tions']
['t', '##el', '##ec', '##om', '##mun', '##ication', '##s']


But, The pieces are in the Turkish model

In [6]:
[t in tokenizer_tur.vocab for t in tokenizer_tur.tokenize(word_en)]

[True, True, True, True, True, True]

In [9]:
long_word_tur = "Muvaffakiyetsizleştiricileştiriveremeyebileceklerimizdenmişsinizcesine"

'''
It means that “As though you happen to have been from among those whom we will not be able to easily/quickly make a maker of unsuccessful ones” 
'''

In [10]:
print(tokenizer_tur.tokenize(long_word_tur))

['muvaffak', '##iyet', '##siz', '##les', '##tir', '##ici', '##les', '##tir', '##iver', '##emeye', '##bilecekleri', '##mi', '##z', '##den', '##mis', '##siniz', '##cesine']


## Understanding Tokenization Algorithms

### Train tokenizers from scratch

let's load Shakespeare plays from gutenberg project

In [11]:
import nltk 
from nltk.corpus import gutenberg 
nltk.download('gutenberg') 
nltk.download('punkt') 
plays=['shakespeare-macbeth.txt','shakespeare-hamlet.txt','shakespeare-caesar.txt']
shakespeare=[" ".join(s) for ply in plays for s in gutenberg.sents(ply)]

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# We prepare a template for the post-processing 
# Some initial settings

In [12]:
from tokenizers.processors import TemplateProcessing
special_tokens= ["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
temp_proc= TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", special_tokens.index("[CLS]")),
        ("[SEP]", special_tokens.index("[SEP]")),
    ],
)

## Training BPE

In [13]:
from tokenizers import Tokenizer
from tokenizers.normalizers import (Sequence,Lowercase, NFD, StripAccents)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.decoders import BPEDecoder

# Instantiate BPE (Byte-Pair Encoding)
tokenizer = Tokenizer(BPE())

# a unicode normalizer, lowercasing and , replacing accents in order  :
# * Sequence : It composes multiple PreTokenizer that will be run in the given order
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])

# Whitespace: Splits on word boundaries using the regular expression \w+|[^\w\s]+ 
tokenizer.pre_tokenizer = Whitespace() 
tokenizer.decoder = BPEDecoder()
tokenizer.post_processor=temp_proc

We are ready to train the model 

In [14]:
from tokenizers.trainers import BpeTrainer
trainer = BpeTrainer(vocab_size=5000, special_tokens= special_tokens)
tokenizer.train_from_iterator(shakespeare, trainer=trainer)
print(f"Trained vocab size: {tokenizer.get_vocab_size()}" )

Trained vocab size: 5000


In [15]:
# take a sentence from macbeth

In [16]:
sen= "Is this a dagger which I see before me, the handle toward my hand?"
sen_enc=tokenizer.encode(sen)
print(f"Output: {format(sen_enc.tokens)}")

Output: ['[CLS]', 'is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', 'le', 'toward', 'my', 'hand', '?', '[SEP]']


In [17]:
sen_enc2=tokenizer.encode("Macbeth and Hugging Face")

In [18]:
print(f"Output: {format(sen_enc2.tokens)}")

Output: ['[CLS]', 'macbeth', 'and', 'hu', 'gg', 'ing', 'face', '[SEP]']


In [19]:
# Let us pass  two sentences

In [20]:
two_enc=tokenizer.encode("I like Hugging Face!","He likes Macbeth!")

In [21]:
print(f"Output: {format(two_enc.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'ing', 'face', '!', '[SEP]', 'he', 'likes', 'macbeth', '!', '[SEP]']


In [22]:
tokenizer.model.save('.')

['./vocab.json', './merges.txt']

In [23]:
!wc -l ./merges.txt

4948 ./merges.txt


In [24]:
!head -6 ./merges.txt

#version: 0.2 - Trained by `huggingface/tokenizers`
t h
o u
a n
th e
r e


In [25]:
!head -1000 ./merges.txt| tail -5

ch ance
si g
your s
ti a
po int


In [26]:
# Save and Load Tokenizer

In [27]:
tokenizer.save("MyBPETokenizer.json")
tokenizerFromFile=Tokenizer.from_file("MyBPETokenizer.json")
sen_enc3 = tokenizerFromFile.encode("I like HuggingFace and Macbeth")
print(f"Output: {format(sen_enc3.tokens)}")

Output: ['[CLS]', 'i', 'like', 'hu', 'gg', 'ing', 'face', 'and', 'macbeth', '[SEP]']


## Training WordPiece

In [28]:
from tokenizers.models import WordPiece
from tokenizers.decoders import WordPiece as WordPieceDecoder
from tokenizers.normalizers import BertNormalizer 

#BERT normalizer includes cleaning the text, handling accents, chinese chars and lowercasing

tokenizer = Tokenizer(WordPiece())
tokenizer.normalizer=BertNormalizer()
tokenizer.pre_tokenizer = Whitespace()

tokenizer.decoder= WordPieceDecoder()

In [29]:
from tokenizers.trainers import WordPieceTrainer
trainer = WordPieceTrainer(vocab_size=5000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tokenizer.train_from_iterator(shakespeare, trainer=trainer)
output = tokenizer.encode(sen)
print(output.tokens)

['is', 'this', 'a', 'dagger', 'which', 'i', 'see', 'before', 'me', ',', 'the', 'hand', '##le', 'toward', 'my', 'hand', '?']


In [30]:
# let us use WordPiece Decoder to treat the sentences properly.

In [31]:
tokenizer.decode(output.ids)

'is this a dagger which i see before me, the handle toward my hand?'

In [32]:
# force the model to produce UNK tokens

In [33]:
tokenizer.encode("Kralsın aslansın Macbeth!").tokens

['[UNK]', '[UNK]', 'macbeth', '!']

# Pre-made tokenizers 
* CharBPETokenizer: The original BPE
* ByteLevelBPETokenizer: The byte level version of the BPE
* SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
* BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece

In [34]:
# Fast Tokenizers optimized for Research and Production

In [35]:
from tokenizers import (ByteLevelBPETokenizer,
                            CharBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)

In [36]:
tokenizer= SentencePieceBPETokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

None


In [37]:
tokenizer= BertWordPieceTokenizer()
print(tokenizer.normalizer)
print(tokenizer.pre_tokenizer)
print(tokenizer.decoder)
print(tokenizer.post_processor)

None
